# Trabajo integrador - Parte 1
## Python y Numpy

**Nombre**: Javier Cambiasso

In [2]:
import numpy as np
import pandas as pd

## Ejercicio 1

Dada una matriz en formato *numpy array*, donde cada fila de la matriz representa un vector matemático, se requiere computar las normas $l_0$, $l_1$, $l_2$, $l_{\infty}$, según la siguientes definiciones:

\begin{equation}
    ||\mathbf{x}||^{p} = \bigg(\sum_{j=1}^{n}{|x_i|^p}\bigg)^{\frac{1}{p}}
\end{equation}

con los casos especiales para $p=0$ y $p=\infty$ siendo:

\begin{equation}
    \begin{array}{rcl}
        ||\mathbf{x}||_0 & = & \bigg(\sum_{j=1 \wedge x_j != 0}{|x_i|}\bigg)\\
        ||\mathbf{x}||_{\infty} & = & \max_{i}{|x_i|}\\
    \end{array}
\end{equation}

In [3]:
def norm(arr,p):
    if p==0:
        return np.count_nonzero(arr,axis=-1)
    elif p==float('inf'):
        return np.max(np.abs(arr),axis=-1)
    else:
        return (np.sum(np.power(np.abs(arr),p),axis=-1))**(1/p)

test_array=np.array([[1,2],[3,4],[5,6]])
print(norm(test_array,0))
print(norm(test_array,10))
print(norm(test_array,float('inf')))

[2 2 2]
[2.00019523 4.02197415 6.09050607]
[2 4 6]


## Ejercicio 2

En clasificación contamos con dos arreglos, la “verdad” y la “predicción”. Cada elemento de los arreglos pueden tomar dos valores, “True” (representado por 1) y “False” (representado por 0). Entonces podemos definir 4 variables:

* True Positive (TP): El valor verdadero es 1 y el valor predicho es 1
* True Negative (TN): El valor verdadero es 0 y el valor predicho es 0
* False Positive (FP): El valor verdadero es 0 y el valor predicho es 1
* False Negative (FN): El valor verdadero es 1 y el valor predicho es 0

A partir de esto definimos:

* Precision = TP / (TP + FP)
* Recall = TP / (TP + FN)
* Accuracy = (TP + TN) / (TP + TN + FP + FN)
 
Calcular las 3 métricas con Numpy y operaciones vectorizadas.

In [12]:
truth = np.array([1,1,0,1,1,1,0,0,0,1])
prediction = np.array([1,1,1,1,0,0,1,1,0,0])

FN = np.count_nonzero(truth > prediction)
FP = np.count_nonzero(prediction > truth)
TP = np.count_nonzero(np.logical_and(prediction,truth))
TN = np.count_nonzero(np.logical_not(np.logical_or(prediction,truth)))

precision = TP/(TP+FP)
recall = TP / (TP + FN)
accuracy = (TP + TN)/(TP + TN + FP + FN)

pd.DataFrame(data={'Precision':precision,'Recall':recall,'Accuracy':accuracy},index=[0])


,Precision,Recall,Accuracy
0,0.5,0.5,0.4


## Ejercicio 3

Crear una función que separe los datos en train-validation-test. Debe recibir de parametros:

- X: Array o Dataframe que contiene los datos de entrada del sistema.
- y: Array o Dataframe que contiene la(s) variable(s) target del problema.
- train_percentage: _float_ el porcentaje de training.
- test_percentage: _float_ el porcentaje de testing.
- val_percentage: _float_ el porcentaje de validación.
- shuffle: _bool_ determina si el split debe hacerse de manera random o no.

Hints: 

* Usar Indexing y slicing
* Usar np.random.[...]

In [10]:
import math
from sklearn.datasets import load_iris

def split(X_input,
          Y_input,
          train_size=0.7,
          val_size=0.15,
          test_size=0.15,
          random_state=42,
          shuffle=True):

    if type(X_input)=='pandas.core.frame.DataFrame':
      X_input = X_input.values

    if type(Y_input)=='pandas.core.frame.DataFrame':
       Y_input = Y_input.values

    if X_input.shape[0] != Y_input.shape[0]: raise TypeError('Dimension 0 for X_input and Y_input must be equal')
    if train_size + val_size + test_size != 1.0: raise ArithmeticError('Sum of train, validation and test ratios must be equal to 1')
    if shuffle:
      rnd = np.random.default_rng(random_state)
      p = rnd.permutation(X_input.shape[0])
      X_input = X_input[p]
      Y_input = Y_input[p]
    
    train_samples = math.floor(X_input.shape[0]*train_size)
    test_samples = math.floor((X_input.shape[0] - train_samples)*(test_size/(1-train_size)))
    
    split_x = np.split(X_input,np.array([0,test_samples])+train_samples,axis=0)
    split_y = np.split(Y_input,np.array([0,test_samples])+train_samples,axis=0)

    X_train = split_x[0]
    Y_train = split_y[0]
    X_test = split_x[1]
    Y_test = split_y[1]
    X_val= split_x[2]
    Y_val = split_y[2]

    return X_train, Y_train, X_test, Y_test, X_val, Y_val

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

X,Y = get_iris_dataset()
X_train, Y_train, X_test, Y_test, X_val, Y_val = split(X,Y,random_state=12)
print(X_train.shape,X_test.shape,X_val.shape)
print(X_train[range(5)])
print(Y_train[range(5)])

(105, 4) (22, 4) (23, 4)
[[6.6 2.9 4.6 1.3]
 [7.2 3.  5.8 1.6]
 [6.  2.9 4.5 1.5]
 [7.2 3.6 6.1 2.5]
 [7.  3.2 4.7 1.4]]
[['versicolor']
 ['virginica']
 ['versicolor']
 ['virginica']
 ['versicolor']]
